In [0]:
sc

In [0]:
pip install -Iv tweepy==3.10.0
pip install textblob

Python interpreter will be restarted.
Using pip 21.0.1 from /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa6aba59-68dc-4989-b588-b4efeaff459f/lib/python3.8/site-packages/pip (python 3.8)
Non-user install because user site-packages disabled
Created temporary directory: /tmp/pip-ephem-wheel-cache-hg56szmf
Created temporary directory: /tmp/pip-req-tracker-yufyar_a
Initialized build tracking at /tmp/pip-req-tracker-yufyar_a
Created build tracker: /tmp/pip-req-tracker-yufyar_a
Entered build tracker: /tmp/pip-req-tracker-yufyar_a
Created temporary directory: /tmp/pip-install-4zhywm0g
1 location(s) to search for versions of tweepy:
* https://pypi.org/simple/tweepy/
Fetching project page and analyzing links: https://pypi.org/simple/tweepy/
Getting page https://pypi.org/simple/tweepy/
Found index url https://pypi.org/simple
Looking up "https://pypi.org/simple/tweepy/" in the cache
Request header has "max_age" as 0, cache bypassed
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:44

In [0]:
import json
import socket
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener


class TweetsListener(StreamListener):
    # tweet object listens for the tweets
    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):

        try:
            msg = json.loads(data)
            print("new message")
            # if tweet is longer than 140 characters
            if "extended_tweet" in msg:
                # add at the end of each tweet "t_end"
                self.client_socket \
                    .send(str(msg['extended_tweet']['full_text'] + "t_end") \
                          .encode('utf-8'))
                print(msg['extended_tweet']['full_text'])
            else:
                # add at the end of each tweet "t_end"
                self.client_socket \
                    .send(str(msg['text'] + "t_end") \
                          .encode('utf-8'))
                print(msg['text'])
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True


def sendData(c_socket, keyword):
    consumer_key = '8xZjcoaxuVJ84lX10LQYVN9q8'
    consumer_secret = 'S6AhI46bG7CrJZ4a49EzXYnfbqnSmXFfuM2FGDfb6lziCihVfc'
    access_token = '1462005344071266306-Av1AALQFaKRAaL2xDTEtg8mXCO7mNS'
    access_secret = 'fL82WS7zBnVxU2YA9OJdAoaVaOF71Zc2FyBItBLm3KchD'

    print('start sending data from Twitter to socket')
    # authentication based on the credentials
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # start sending data from the Streaming API
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=keyword, languages=["en"])


if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "127.0.0.1"
    port = 5556
    s.bind((host, port))
    print('socket is ready')
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    # select here the keyword for the tweet data
    sendData(c_socket, keyword=['piano'])


socket is ready
socket is listening


Python interpreter will be restarted.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa6aba59-68dc-4989-b588-b4efeaff459f/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
from textblob import TextBlob


def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words


# text classification
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity


def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity


def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("word"))
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
    return words


if __name__ == "__main__":
    # create Spark session
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").config("spark.executor.heartbeatInterval","3600s").config("spark.network.timeout", "3601s").config("spark.files.fetchTimeout", "3600s").getOrCreate()


    # read the tweet data from socket
    lines = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 5556).option("soTimeout", 20000).load()
    # Preprocess the data
    words = preprocessing(lines)
    # text classification to define polarity and subjectivity
    words = text_classification(words)

    words = words.repartition(1)
    query = words.writeStream.queryName("all_tweets") \
        .outputMode("append").format("parquet") \
        .option("path", "dbfs:/") \
        .option("checkpointLocation", "dbfs:/") \
        .trigger(processingTime='60 seconds').start()
    query.awaitTermination()


In [0]:
%fs ls 



In [0]:
df = spark.read.parquet('dbfs:/part-00000-02de3ac4-c4a7-4514-8c83-a892a8d00b52-c000.snappy.parquet')

In [0]:
df.head()